In [ ]:
from io import StringIO
from wcf.records import Record, print_records
from contextlib import redirect_stdout
import xml.dom.minidom
import pandas as pd
import numpy as np
from pathlib import Path


def xmlparse(xmlstr):
    '''
    parse air quality xml data to dict list
    '''
    dom = xml.dom.minidom.parseString(xmlstr)
    stats = dom.getElementsByTagName("AQIDataPublishLive")
    airdata = []
    for stat in stats:
        r = {}
        for node in stat.childNodes:
            if (node.nodeName == "#text" or
                    node.nodeName == "OpenAccessGenerated"):
                continue
            inx = node.nodeName
            inx = inx.lower()
            for n in node.childNodes:
                r[inx] = n.data
        airdata.append(r)
    return airdata


def data_from_xml_json(xmlfile):
    '''
    预处理由 wcf 转换后的 xml，将不必要的字符删除
    '''
    data = xmlfile

    # this is for the air quality data, to split some charicters
    data = data.replace("a:", "")
    data = data.replace("b:", "")
    data = data.replace("c:", "")
    data = data.replace("&mdash", "-")
    return xmlparse(data)

### 读取原始 wcf 数据列表

In [ ]:
# 尽量用后一个时刻的结果（也就是第 13 分钟），可能数据更全些
fileList = list(Path('.').glob('cnemc_*13'))

In [ ]:
# fileList = list(Path('.').glob('cnemc_*43'))

In [ ]:
# 如果有过小的文件则直接删除，并重新运行下上面的文件列表
for i, f in enumerate(fileList):
    if f.stat().st_size <= 264:
        print(i, f, f.stat().st_size)
        !mv {f} error

### 转换 wcf 为 csv
如遇到问题，请重新读取数据列表，同时手动对`err_num`加1

In [ ]:
err_num = 0
for f in fileList[err_num:]:
    print(f)
    r = open(f, 'rb')
    records = Record.parse(r)

    # 将print在std.out的内容赋予变量out_xml
    ff = StringIO()
    with redirect_stdout(ff):
        print_records(records)
    out_xml = ff.getvalue()

    # 将数据从总的 xml 转为各个 dict 组成的 list
    data_dict = data_from_xml_json(out_xml)

    # 将字典转为 DataFrame
    df = pd.DataFrame.from_dict(data_dict)
    # 如果出现不同的时间就先输出一份原始数据
    if len(df['timepoint'].unique()) > 1:
        df.to_csv('Error'+df['timepoint'].unique()[0][:13]+'.csv', index=None)

    # 将数据处理下再输出
    df_ = df[['timepoint', 'stationcode', 'longitude', 'latitude',
              'area', 'positionname', 'primarypollutant', 'aqi',
              'pm10', 'pm10_24h', 'pm2_5', 'pm2_5_24h',
              'o3', 'o3_24h', 'o3_8h', 'o3_8h_24h',
              'no2', 'no2_24h', 'so2', 'so2_24h',
              'co', 'co_24h']]
    df_ = df_.where(df != '-;', np.nan)
    # 将每小时数据保存为 csv 文件
    df_.to_csv(df['timepoint'].unique()[-1][:13]+'.csv', index=None)
    # 没问题就可以删除
    f.unlink()

### 压缩 csv 文件
压缩文件名为当前时间

In [ ]:
!zip cnemc13_$(date +%Y%m%d%H%M).zip *.csv
!rm *.csv
!mv cnemc*13 error

In [ ]:
# zip cnemc43_$(date +%Y%m%d%H%M).zip *.csv
# !rm *.csv
# !mv cnemc*43 Error

### 上传、归档压缩文件

In [ ]:
!../transfer cow *.zip -s
!mv *.zip Archive